# Modelo para Inflação dos Alimentos 

O modelo proposto pelo BCB em 2016 para modelar a inflação de alimentos consistia na seguinte equação

$$\pi_t^{AD} = \sum_{i=0}^1 \phi_i \; \Delta \epsilon_{t-i} + \sum_{i=0}^1 \beta_i \;\Delta IC_{t-i}^{agro}+ \delta \; oni_{t-1} + \gamma \; h_t + \sum_{i=1}^4 \alpha_i \; T_i + \varepsilon_t$$

onde $\pi_t^{AD}$ é inflação de alimentação no domicílio (% 4 trimestres), $\Delta\epsilon_{t}$ é a variação da taxa de câmbio nominal, $\Delta IC_{t-i}^{agro}$ é a variação do segmento de commodities agropecuárias do Índice de Commodities - Brasil (medido em dólares), $oni_{t-1}$ é a primeira defasagem da dummy se o mundo está ou não sob efeito do El Niño, $h_t$ é o hiato do produto e $T_i$ são dummies sazonais.


In [1]:
!pip install linearmodels

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.7 MB 6.1 MB/s eta 0:00:01
   --------------------- ------------------ 0.9/1.7 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.7 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 11.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/94.2 kB ? eta -:--:--
   ---------------------------------------- 94.2/94.2 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
   ---------------------------------------- 43.7/43.7 kB 2.1 MB/s eta 0:00:00


# Bibliotecas 

In [2]:
from bcb import sgs
import pandas as pd
import numpy as no
import sidrapy as sidra
from functools import reduce
from linearmodels.iv import IV2SLS

# Coletando dados IPCA Variação mensal - subgrupos alimentação domicílios

In [4]:
# Coleta de dados
dados_brutos = list(
    map(
        # função com 2 argumentos que será repetida
        lambda tabela, variavel: ( # lambda argumento1, argumento2: função/expressão a ser executada
            sidra.get_table(
                table_code = tabela,
                territorial_level = "1", # alguns argumentos recebem valores padrão
                ibge_territorial_code = "all",
                variable = variavel,
                classifications = { # IPCA - Variação mensal - subgrupos alimentação no domicílio
                    "315": "7171"
                    },
                period = "all",
                header = 'n'
                )
            ),

        # códigos das tabelas (pro argumento tabela)
        # agosto/1999 até junho/2006
        # julho/2006 até dezembro/2011
        # janeiro/2012 até dezembro/2019
        # janeiro/2020 >
        ["655", "2938", "1419", "7060"],

        # códigos da variável dentro da tabela (pro argumento variavel)
        ["all", "63", "63", "63"]
        )
    )
dados_brutos # lista de DataFrames

[   NC      NN MC MN      V D1C     D1N     D2C             D2N D3C  \
 0   1  Brasil  2  %   0.17   1  Brasil  199908     agosto 1999  63   
 1   1  Brasil  2  %   0.42   1  Brasil  199909   setembro 1999  63   
 2   1  Brasil  2  %   2.30   1  Brasil  199910    outubro 1999  63   
 3   1  Brasil  2  %   1.66   1  Brasil  199911   novembro 1999  63   
 4   1  Brasil  2  %   1.55   1  Brasil  199912   dezembro 1999  63   
 .. ..     ... .. ..    ...  ..     ...     ...             ...  ..   
 78  1  Brasil  2  %  -0.52   1  Brasil  200602  fevereiro 2006  63   
 79  1  Brasil  2  %  -0.38   1  Brasil  200603      março 2006  63   
 80  1  Brasil  2  %  -0.51   1  Brasil  200604      abril 2006  63   
 81  1  Brasil  2  %  -0.22   1  Brasil  200605       maio 2006  63   
 82  1  Brasil  2  %  -0.92   1  Brasil  200606      junho 2006  63   
 
                        D3N   D4C                          D4N  
 0   IPCA - Variação mensal  7171  11.Alimentação no domicílio  
 1   IPCA - Vari

In [5]:
# Função para acumular os trimestres
def acum_quarter(x):
    """
    Acumula a variação percentual mensal em um trimestre.

    Argumentos:
    - x: lista, array ou série pandas contendo os valores de inflação.

    Retorno:
    - A inflação acumulada em um trimestre, em porcentagem.
    """

    # Transforma os valores em fatores de crescimento
    x_fac = 1 + (x / 100)

    # Calcula o produto acumulado dos últimos três valores
    x_cum = np.prod(x_fac[-3:])

    # Calcula o valor acumulado em um trimestre
    x_qr = (x_cum - 1) * 100

    return x_qr

In [6]:
import numpy as np

def acum_p(data, n):
    """
    Calcula a acumulação de variações percentuais em janelas móveis.

    Argumentos:
    - data: array-like
        Uma sequência de dados representando variações percentuais.

    - n: int
        O tamanho da janela móvel para o cálculo da acumulação.

    Retorno:
    - array
        Um array contendo a acumulação das variações percentuais em janelas móveis.
    """

    resultado = (((data / 100) + 1)
                 .rolling(window=n)
                 .apply(np.prod)
                 - 1) * 100
    return resultado


In [7]:
# Tratamento
alim =  (
    pd.concat(  # empilha em uma tabela todos os DataFrames
        objs = dados_brutos,
        names = ["tabela", "linha"]
        )
    .reset_index()
    # filtra as colunas
    .filter(items = ["D2C", "V"], axis = "columns")
    # altera os tipos das colunas
    .assign(
        data = lambda x: pd.to_datetime(x.D2C, format = "%Y%m"),
        valor = lambda x: x.V.astype(float)
        )
    # filtra as colunas
    .filter(items = ["data", "valor"], axis = "columns")
)

In [8]:
alim

,data,valor
0,1999-08-01,0.17
1,1999-09-01,0.42
2,1999-10-01,2.30
3,1999-11-01,1.66
4,1999-12-01,1.55
...,...,...
292,2023-12-01,1.34
293,2024-01-01,1.81
294,2024-02-01,1.12
295,2024-03-01,0.59


In [17]:
alim_q = (   alim
    # cria colunas de trimestre
    .assign(date_quarter = lambda x: pd.PeriodIndex(x['data'], freq = 'Q'))
    # agrupa por trimestre e acumula os valores percentuais do período
    .groupby(by = 'date_quarter')
    .agg({
        'valor': lambda x: acum_quarter(x)
    })
    .reset_index()
    # calcula o produto móvel de 4 trimestres
    .assign(alimentos = lambda x: acum_p(x.valor, n = 4))
    )

In [18]:
alim_q

,date_quarter,valor,alimentos
0,1999Q3,0.590714,NaN
1,1999Q4,5.610152,NaN
2,2000Q1,-0.197442,NaN
3,2000Q2,-1.445768,4.491390
4,2000Q3,5.662035,9.759364
...,...,...,...
95,2023Q2,-0.347811,2.878549
96,2023Q3,-2.970825,-0.780469
97,2023Q4,2.375720,-0.516884
98,2024Q1,3.557679,2.510526


Índice de Commodities - Brasil - Agropecuária (em US Dólares)
Taxa de câmbio - Livre - Dólar americano (compra) - Média de período - mensal

In [19]:
# Coleta e tratamento
## Índice de Commodities - Brasil - Agropecuária
## Taxa de câmbio - Livre - Dólar americano (compra) - Média de período - mensal
ic_dol = (
    # coleta os dados
    sgs.get({'dolar' : 3697,
             'icagro' : 29041},
        start = '1998-01-01')
    .reset_index()
    # cria coluna de trimestre
    .assign(date_quarter = lambda x: pd.PeriodIndex(x['Date'], freq = 'Q'))
    # agrupa por trimestre e calcula a média por trimestre
    .groupby(by = 'date_quarter')
    .agg({
        'dolar': lambda x: np.mean(x),
        'icagro': lambda x: np.mean(x)
    })
    .reset_index()
    # primeira diferença da média trimestral
    .assign(dolar_diff = lambda x: x.dolar.pct_change(),
            icagro_diff = lambda x: x.icagro.pct_change())
    .dropna()
)

ic_dol

,date_quarter,dolar,icagro,dolar_diff,icagro_diff
1,1998Q2,1.147167,88.663333,0.018708,-0.056405
2,1998Q3,1.170567,81.790000,0.020398,-0.077522
3,1998Q4,1.195033,80.003333,0.020902,-0.021845
4,1999Q1,1.770000,75.236667,0.481130,-0.059581
5,1999Q2,1.713533,70.013333,-0.031902,-0.069425
...,...,...,...,...,...
101,2023Q2,4.950767,176.656667,-0.046836,0.008295
102,2023Q3,4.879833,184.576667,-0.014328,0.044833
103,2023Q4,4.952833,180.736667,0.014960,-0.020804
104,2024Q1,4.952367,183.456667,-0.000094,0.015050


## Hiato do Produto (BCB)

In [29]:
# Coleta e tratamento do Hiato do Produto do BC
hiato = (
    pd.read_excel(
    "ri202403anp.xlsx",
    sheet_name = "Graf 2.2.4",
    skiprows = 8
    )
    .assign(date_quarter = lambda x: pd.PeriodIndex(x['Trimestre'], freq = 'Q'),
            hiato = lambda x: x.Hiato.astype(float))
    .loc[:, ['date_quarter', 'hiato']]
  )

In [30]:
hiato

,date_quarter,hiato
0,NaT,NaN
1,2003Q4,-0.83
2,2004Q1,-0.40
3,2004Q2,-0.03
4,2004Q3,0.35
...,...,...
78,2023Q1,-0.61
79,2023Q2,-0.64
80,2023Q3,-0.66
81,2023Q4,-0.60


In [31]:
# Importa os dados da página
oni = pd.read_csv('https://www.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/detrend.nino34.ascii.txt', sep='\s+')

# Certifica que estejam no tipo numérico
oni = oni.apply(pd.to_numeric, errors='coerce')

# Retira dados faltantes
oni.dropna(inplace = True)

In [32]:
oni

,YR,MON,TOTAL,ClimAdjust,ANOM
0,1950,1,24.56,26.18,-1.62
1,1950,2,25.07,26.39,-1.32
2,1950,3,25.88,26.95,-1.07
3,1950,4,26.29,27.39,-1.11
4,1950,5,26.19,27.56,-1.37
...,...,...,...,...,...
888,2024,1,28.37,26.55,1.82
889,2024,2,28.28,26.76,1.52
890,2024,3,28.42,27.29,1.13
891,2024,4,28.61,27.83,0.79


In [33]:
# Tratamento
df_oni = (
  oni
  # renomeia as colunas
 .rename(columns =
          {'YR' : 'Year',
           'MON' : 'Month'})
 .assign(oni = lambda x : x.ANOM.rolling(window = 3, center = True).mean())
  # cria coluna de data trimestral
 .assign(Quarter = lambda x: pd.to_datetime(x[['Year', 'Month']].assign(Day = 1)),
         date_quarter = lambda x: pd.PeriodIndex(x['Quarter'], freq = 'Q'))
  # agrupa por trimestre
  .groupby(by = 'date_quarter')
  # calcula a média
  .agg(oni = ('oni', 'mean'))
  .reset_index()
)

# Adicionar a condição com base na condição oni > 0.5
df_oni['nino'] = df_oni['oni'].apply(lambda x: 1 if x > 0.5 else 0)

In [34]:
df_oni

,date_quarter,oni,nino
0,1950Q1,-1.251667,0
1,1950Q2,-1.035556,0
2,1950Q3,-0.446667,0
3,1950Q4,-0.614444,0
4,1951Q1,-0.508889,0
...,...,...,...
293,2023Q2,0.470000,0
294,2023Q3,1.316667,1
295,2023Q4,1.884444,1
296,2024Q1,1.474444,1


## Junta os dados e cria variáveis (defasagens e dummies)

In [35]:
# lista de dataframes
dfs = [alim_q, ic_dol, hiato, df_oni]

# reduz os dataframes pela chave 'date_quarter' com left join
dados_reg = reduce(lambda left, right: pd.merge(left, right, on = 'date_quarter', how = 'left'), dfs).dropna()
dados_reg['dolar_diff_lag'] = dados_reg['dolar_diff'].shift(1)
dados_reg['icagro_diff_lag'] = dados_reg['icagro_diff'].shift(1)

In [36]:
dados_reg

,date_quarter,valor,alimentos,dolar,icagro,dolar_diff,icagro_diff,hiato,oni,nino,dolar_diff_lag,icagro_diff_lag
17,2003Q4,0.852288,6.721160,2.899400,82.116667,-0.011871,0.076752,-0.83,0.328889,0,NaN,NaN
18,2004Q1,1.252923,2.248280,2.895067,80.926667,-0.001495,-0.014492,-0.40,0.304444,0,-0.011871,0.076752
19,2004Q2,0.346135,1.763190,3.044367,86.276667,0.051570,0.066109,-0.03,0.206667,0,-0.001495,-0.014492
20,2004Q3,1.090541,3.586819,2.976133,82.773333,-0.022413,-0.040606,0.35,0.604444,1,0.051570,0.066109
21,2004Q4,-0.142363,2.565198,2.784900,84.076667,-0.064256,0.015746,0.63,0.672222,1,-0.022413,-0.040606
...,...,...,...,...,...,...,...,...,...,...,...,...
94,2023Q1,0.499344,7.037006,5.194033,175.203333,-0.011639,0.011138,-0.61,-0.418889,0,0.001207,0.001194
95,2023Q2,-0.347811,2.878549,4.950767,176.656667,-0.046836,0.008295,-0.64,0.470000,0,-0.011639,0.011138
96,2023Q3,-2.970825,-0.780469,4.879833,184.576667,-0.014328,0.044833,-0.66,1.316667,1,-0.046836,0.008295
97,2023Q4,2.375720,-0.516884,4.952833,180.736667,0.014960,-0.020804,-0.60,1.884444,1,-0.014328,0.044833


In [37]:
# Criar variáveis dummy trimestrais usando pd.get_dummies
dummies = pd.get_dummies(dados_reg['date_quarter'].dt.quarter, prefix = 'trimestre', dtype = int)

In [38]:
# Concatenar as variáveis dummy com o DataFrame original
dados_reg = pd.concat([dados_reg, dummies], axis=1).dropna()

In [39]:
dados_reg

,date_quarter,valor,alimentos,dolar,icagro,dolar_diff,icagro_diff,hiato,oni,nino,dolar_diff_lag,icagro_diff_lag,trimestre_1,trimestre_2,trimestre_3,trimestre_4
18,2004Q1,1.252923,2.248280,2.895067,80.926667,-0.001495,-0.014492,-0.40,0.304444,0,-0.011871,0.076752,1,0,0,0
19,2004Q2,0.346135,1.763190,3.044367,86.276667,0.051570,0.066109,-0.03,0.206667,0,-0.001495,-0.014492,0,1,0,0
20,2004Q3,1.090541,3.586819,2.976133,82.773333,-0.022413,-0.040606,0.35,0.604444,1,0.051570,0.066109,0,0,1,0
21,2004Q4,-0.142363,2.565198,2.784900,84.076667,-0.064256,0.015746,0.63,0.672222,1,-0.022413,-0.040606,0,0,0,1
22,2005Q1,1.132977,2.443697,2.664367,87.613333,-0.043281,0.042065,0.65,0.555556,1,-0.064256,0.015746,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2023Q1,0.499344,7.037006,5.194033,175.203333,-0.011639,0.011138,-0.61,-0.418889,0,0.001207,0.001194,1,0,0,0
95,2023Q2,-0.347811,2.878549,4.950767,176.656667,-0.046836,0.008295,-0.64,0.470000,0,-0.011639,0.011138,0,1,0,0
96,2023Q3,-2.970825,-0.780469,4.879833,184.576667,-0.014328,0.044833,-0.66,1.316667,1,-0.046836,0.008295,0,0,1,0
97,2023Q4,2.375720,-0.516884,4.952833,180.736667,0.014960,-0.020804,-0.60,1.884444,1,-0.014328,0.044833,0,0,0,1


# 3. Estimação

In [40]:
# Regressão linear
reg = IV2SLS.from_formula(
    formula = "alimentos ~   - 1 + dolar_diff + dolar_diff_lag + icagro_diff + icagro_diff_lag + hiato + nino + trimestre_1 + trimestre_2 + trimestre_3",  # especificação do modelo no formato de fórmula
    data = dados_reg  # fonte dos dados
    ).fit(cov_type="unadjusted")

In [41]:
print(reg.summary)

                            OLS Estimation Summary                            
Dep. Variable:              alimentos   R-squared:                      0.5290
Estimator:                        OLS   Adj. R-squared:                 0.4701
No. Observations:                  81   F-statistic:                    90.978
Date:                Thu, Jun 06 2024   P-value (F-stat)                0.0000
Time:                        12:06:19   Distribution:                  chi2(9)
Cov. Estimator:            unadjusted                                         
                                                                              
                                Parameter Estimates                                
                 Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------
dolar_diff          26.840     11.956     2.2449     0.0248      3.4069      50.273
dolar_diff_lag      24.564     1